In [1]:
import pandas as pd
from functools import partial

In [2]:
def get_data(area_type, release, earliest=None, latest=None):
    data = pd.read_csv(
        f'https://api.coronavirus.data.gov.uk/v2/data?areaType={area_type}'
        f'&metric=newCasesBySpecimenDate&format=csv&release={release}',
    )
    if earliest:
        data = data[data['date'] >= earliest]
    if latest:
        data = data[data['date'] <= latest]
    return data.groupby('date').sum().sum()

In [3]:
def diff(area_type, new_release, old_release, earliest=None, latest=None):
    new_data = get_data(area_type, new_release, earliest, latest=latest)
    old_data = get_data(area_type, old_release, earliest, latest=latest)
    return new_data-old_data

In [4]:
diff_ = partial(diff, new_release='2022-01-31', old_release='2022-01-30')

In [5]:
def diff__(earliest):
    return (diff_('nation', earliest=earliest) - diff_('region', earliest=earliest)).squeeze()

The difference comes from where the series is truncated. For the `nation` series, the data was truncated at 25th July '2021: 

In [6]:
diff_('nation',  earliest='2021-07-25')

newCasesBySpecimenDate    731988
dtype: int64

Meanwhile, the `region` series went all the way back to the 1st April 2020:

In [7]:
region_diff = diff_('region', earliest='2020-04-01')
region_diff

newCasesBySpecimenDate    814099
dtype: int64

If we consider the same time ranges then there are still differences between the `nation` and `region` *areaType*, but they're much less dramatic:

In [8]:
print(f"whole series: {diff__(earliest=None):,}")
print(f"to 1st April 20: {diff__(earliest='2020-01-04'):,}")
print(f"to 25th July 21: {diff__(earliest='2021-07-25'):,}")

whole series: 37,163
to 1st April 20: 37,163
to 25th July 21: 25,250


Interesting that the 31st Jan release appears to have removed some cases that had been reported for specimen dates earlier than April 1st: 601 cases at a national level and 326 cases at a regional level!

In [9]:
diff_('nation', latest='2020-04-01')

newCasesBySpecimenDate   -601
dtype: int64

In [10]:
diff_('region', latest='2020-04-01')

newCasesBySpecimenDate   -326
dtype: int64